# 1.Setup

## Installation

We would require ther following additional packages:
1. google-cloud-storage==2.17.0
2. google-cloud-documentai==2.29.1
3. google-cloud-aiplatform==1.49.0
4. vertexai==1.49.0

In [ ]:
!pip install --upgrade --quiet google-cloud-storage==2.17.0
!pip install --upgrade --quiet google-cloud-documentai==2.29.1
!pip install --upgrade --quiet google-cloud-aiplatform==1.49.0
!pip install --upgrade --quiet vertexai==1.49.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 38.6 MB/s eta 0:00:00


## Create a GCP project and setup configs for it.

In [86]:
!gcloud auth application-default login
!gcloud config set project rag-llmops-gcp
!gcloud config get-value project

In [87]:
# Run the following code if the colab notebook performs incomplete authentication.
# !gcloud auth login

In [88]:
PROJECT_ID_DETAILS = !gcloud config get-value project
PROJECT_ID = PROJECT_ID_DETAILS[0]
PROJECT_NUMBER_DETAILS = !gcloud projects describe $PROJECT_ID --format="value(projectNumber)"
PROJECT_NUMBER = PROJECT_NUMBER_DETAILS[0]  # The project number is item 0 in the list returned by the gcloud command


# !gsutil mb -l us-central1 gs://{BUCKET}
BUCKET=f"{PROJECT_ID}-bucket"
REGION="northamerica-northeast1"
print(f"Project ID: {PROJECT_ID}")
print(f"Project Number: {PROJECT_NUMBER}")
print(f"Bucket Name: {BUCKET}")

#2.Divide documents in chunks

Get sample doc from git

In [ ]:
# !git clone https://github.com/Sahilvohra58/gen-ai-on-google-cloud.git

Cloning into 'gen-ai-on-google-cloud'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), 685.69 KiB | 9.52 MiB/s, done.


## Create a document AI processor

Useful resources:
- https://cloud.google.com/document-ai/docs/samples/documentai-create-processor?hl=en
- https://cloud.google.com/document-ai/docs/overview#dai-processors
- https://cloud.google.com/python/docs/reference/documentai/1.2.1/google.cloud.documentai_v1beta3.services.document_processor_service.DocumentProcessorServiceClient
- https://cloud.google.com/document-ai/docs/layout-parse-chunk (How to create chunks from docs. Not mentioned in the video)


In [ ]:
from datetime import datetime
from google.api_core.client_options import ClientOptions
from google.cloud import documentai

UID = datetime.now().strftime("%m%d%H%M")

location = 'us'
processor_display_name = f"RAG-Chunking-Processor-{UID}"
processor_type = 'LAYOUT_PARSER_PROCESSOR'

opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")
client = documentai.DocumentProcessorServiceClient(client_options=opts)

parent = client.common_location_path(PROJECT_NUMBER, location)

processor = client.create_processor(
    parent=parent,
    processor=documentai.Processor(
        display_name=processor_display_name, type_=processor_type
    ),
)

print(f"Processor Name: {processor.name}")
print(f"Processor Display Name: {processor.display_name}")
print(f"Processor Type: {processor.type_}")
print(f"Processor State: {processor.state}")

processor_name = processor.name

Processor Name: projects/314626027617/locations/us/processors/5d7d5f3c2d28335
Processor Display Name: RAG-Chunking-Processor-07090208
Processor Type: LAYOUT_PARSER_PROCESSOR
Processor State: 1


## Parse document(s) into chunks

In [ ]:
from typing import Optional
from google.cloud import documentai_v1beta3 as documentai

file_path = "/content/gen-ai-on-google-cloud/Cinnamon.pdf"
mime_type = "application/pdf"

def process_document_sample(
    project_id: str,
    location: str,
    processor_name: str,
    file_path: str,
    mime_type: str,
) -> None:
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    name = processor_name
    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    with open(file_path, "rb") as image:
        image_content = image.read()

    raw_document = documentai.RawDocument(content=image_content, mime_type=mime_type)

    process_options = documentai.ProcessOptions(
        layout_config=documentai.ProcessOptions.LayoutConfig(
            chunking_config=documentai.ProcessOptions.LayoutConfig.ChunkingConfig(
                chunk_size=200,
                include_ancestor_headings=True
            )
        )
    )


    request = documentai.ProcessRequest(
        name=name,
        raw_document=raw_document,
        process_options=process_options,
    )

    result = client.process_document(request=request)
    document = result.document
    return document

document_object = process_document_sample(PROJECT_NUMBER, location, processor_name, file_path, mime_type)

doc_layout = document_object.document_layout
chunked_doc = document_object.chunked_document

In [ ]:
print(doc_layout.blocks[:2])

[block_id: "1"
text_block {
  text: "ASN"
  type_: "paragraph"
}
page_span {
  page_start: 1
  page_end: 1
}
, block_id: "2"
text_block {
  text: "EST. 1928"
  type_: "paragraph"
}
page_span {
  page_start: 1
  page_end: 1
}
]


In [ ]:
text_chunks = [chunk.content for chunk in chunked_doc.chunks]
print(text_chunks[:5])

['ASNEST. 1928AmericanSociety forNutritionExcellence in Nutrition Research and PracticeThe American Journal ofCLINICAL NUTRITIONjournal homepage: https://ajcn.nutrition.org/Themerican JournalCLINICAL NUTRITIONOriginal Research Article', '# Effect of cinnamon spice on continuously monitored glycemic response in adults with prediabetes: a 4-week randomized controlled crossover trial\n\nHila Zelicha¹, Jieping Yang 2, Susanne M Henning 2, Jianjun Huang 2, Ru-Po Lee2, Gail Thames 2, Edward H Livingston¹, David Heber², Zhaoping Li 2,*Department of Surgery, University of California, Los Angeles, Los Angeles, CA, United States; 2 Department of Medicine, Center for Human Nutrition, David Geffen School of Medicine, Los Angeles, United States', '# Effect of cinnamon spice on continuously monitored glycemic response in adults with prediabetes: a 4-week randomized controlled crossover trial\n\n## ABSTRACT\n\nCheck for updatesBackground: Previous clinical studies showing that cinnamon spice lowers b


#3.Convert to embeddings

- https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings
- https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/text-embeddings-api
- https://cloud.google.com/vertex-ai/docs/vector-search/setup/format-structure

In [ ]:
import os
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

!gcloud auth application-default set-quota-project $PROJECT_ID

## Restart kernel after this!!


Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "rag-llmops-gcp" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


## Get embeddings from chunks

In [ ]:
from typing import List

from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel


def embed_text(
    texts: List[str] = text_chunks,  # Use your extracted chunks
    task: str = "RETRIEVAL_DOCUMENT",
    model_name: str = "textembedding-gecko@003",
) -> List[List[float]]:
    model = TextEmbeddingModel.from_pretrained(model_name)
    inputs = [TextEmbeddingInput(text, task) for text in texts]
    embeddings = model.get_embeddings(inputs)
    return [embedding.values for embedding in embeddings]

embeddings = embed_text(text_chunks)

## Export all embeddings to gcs

In [ ]:
import json
filename = f"embeddings-{UID}.json"

def create_embeddings_jsonl(text_chunks, embeddings, filename):
    with open(filename, 'w') as outfile:
        for idx, (text, embedding) in enumerate(zip(text_chunks, embeddings)):
            data = {
                "id": idx,
                "text": text,
                "embedding": embedding
            }
            json.dump(data, outfile, separators=(',', ':'))
            outfile.write('\n')

create_embeddings_jsonl(text_chunks, embeddings, filename)

In [ ]:
embeddings_path = f"gs://{BUCKET}/embeddings-data-{UID}/batch_root/"
!gsutil cp {filename} {embeddings_path}

Copying file://embeddings-07090208.json [Content-Type=application/json]...
-
Operation completed over 1 objects/1.1 MiB.                                      


In [ ]:
embeddings[0][0]

0.021643755957484245

# 4.Create Vector Search index

- https://cloud.google.com/vertex-ai/docs/vector-search/create-manage-index
- https://cloud.google.com/vertex-ai/docs/vector-search/configuring-indexes
- https://cloud.google.com/vertex-ai/docs/vector-search/configuring-indexes#distance-measure-type
- https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.MatchingEngineIndexEndpoint#google_cloud_aiplatform_MatchingEngineIndexEndpoint_deploy_index

In [ ]:
first_embedding = embeddings[0]
embeddings_dimensionality = len(first_embedding)
print(f"Embeddings dimensionality: {embeddings_dimensionality}")

Embeddings dimensionality: 768


##Setup Index

In [ ]:
from google.cloud import aiplatform

def vector_search_create_index(
    project: str, location: str, display_name: str, gcs_uri: Optional[str] = None
) -> None:

    aiplatform.init(project=project, location=location, staging_bucket=BUCKET)


    index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
        display_name=display_name,
        contents_delta_uri=f"{embeddings_path}",
        description="RAG Index",
        dimensions=embeddings_dimensionality,
        approximate_neighbors_count=50,
        leaf_node_embedding_count=500,
        leaf_nodes_to_search_percent=7,
        index_update_method="batch_update",
        distance_measure_type="DOT_PRODUCT_DISTANCE",
    )
    return(index)


display_name = f"RAG-index-{UID}"
vvs_index = vector_search_create_index(PROJECT_ID, REGION, display_name)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Creating MatchingEngineIndex
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Create MatchingEngineIndex backing LRO: projects/314626027617/locations/northamerica-northeast1/indexes/5046494488701173760/operations/3335232733172465664
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:MatchingEngineIndex created. Resource name: projects/314626027617/locations/northamerica-northeast1/indexes/5046494488701173760
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:To use this MatchingEngineIndex in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:index = aiplatform.MatchingEngineIndex('projects/314626027617/locations/northamerica-northeast1/indexes/5046494488701173760')


## Create index endpoint and deploy

- Check the console to see the the deployment process live. The below code might timeout but the console would finish deploying in 1-2 hours.

In [ ]:
vvs_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name = f"index-endpoint-{UID}",
    public_endpoint_enabled = True
)

DEPLOYED_INDEX_ID = f"deployed_index_{UID}"

vvs_index_endpoint.deploy_index(
    index = vvs_index, deployed_index_id = DEPLOYED_INDEX_ID, deploy_request_timeout=2000
)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Creating MatchingEngineIndexEndpoint
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Create MatchingEngineIndexEndpoint backing LRO: projects/314626027617/locations/northamerica-northeast1/indexEndpoints/8155033762749480960/operations/1099195518183014400
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:MatchingEngineIndexEndpoint created. Resource name: projects/314626027617/locations/northamerica-northeast1/indexEndpoints/8155033762749480960
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:To use this MatchingEngineIndexEndpoint in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/314626027617/locations/northamerica-northeast1/indexEndpoints/8155033762749480960')
INFO:google.cloud.aiplatform.matching_engine.matching_engine

TimeoutError: Operation did not complete within the designated timeout of 900 seconds.

#5.Perform a similarity search

## Retrieve the index of nearest neighbours

In [ ]:
def embed_query(text: str, task: str = "RETRIEVAL_DOCUMENT", model_name: str = "textembedding-gecko@003"):
    model = TextEmbeddingModel.from_pretrained(model_name)
    input = TextEmbeddingInput(text, task)
    embeddings = model.get_embeddings([input])
    return embeddings[0].values

query_text = "How might cinnamon supplementation interact with other dietary or lifestyle interventions for prediabetes management?"
query_embedding = embed_query(query_text)

response = vvs_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=[query_embedding],
    num_neighbors=3
)

In [ ]:
print(response)

[[MatchNeighbor(id='25', distance=0.8718024492263794, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]), MatchNeighbor(id='1', distance=0.8679857850074768, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]), MatchNeighbor(id='62', distance=0.866719126701355, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[])]]


##Convert index to embeddings

In [90]:
neighbor_id = response[0][0].id
neighbor_embedding = vvs_index_endpoint.read_index_datapoints(deployed_index_id=DEPLOYED_INDEX_ID, ids= [neighbor_id],)
print(neighbor_embedding)

##Convert Embeddings to texts

In [ ]:
neighbor_ids = [neighbor.id for neighbor in response[0]]


def fetch_text_chunks(ids, filename=filename):
    texts = {}
    with open(filename, 'r') as file:
        for line in file:
            data = json.loads(line)
            if str(data['id']) in ids:  # Ensure the id from JSON is treated as a string for matching
                texts[str(data['id'])] = data['text']
    return [texts[id] for id in ids if id in texts]


neighbor_texts = fetch_text_chunks([str(id) for id in neighbor_ids], filename=filename)


print(neighbor_texts)

['# Effect of cinnamon spice on continuously monitored glycemic response in adults with prediabetes: a 4-week randomized controlled crossover trial\n\n## CGM\n\n### TABLE 1\n\nBaseline characteristics of the study participants (n=18)^{1}', '# Effect of cinnamon spice on continuously monitored glycemic response in adults with prediabetes: a 4-week randomized controlled crossover trial\n\nHila Zelicha¹, Jieping Yang 2, Susanne M Henning 2, Jianjun Huang 2, Ru-Po Lee2, Gail Thames 2, Edward H Livingston¹, David Heber², Zhaoping Li 2,*Department of Surgery, University of California, Los Angeles, Los Angeles, CA, United States; 2 Department of Medicine, Center for Human Nutrition, David Geffen School of Medicine, Los Angeles, United States', '# Appendix A. Supplementary data\n\n## References\n\n### H. Zelicha et al.\n\n[54] M.L. Silva, M.A. Bernardo, J. Singh, M.F. de Mesquita, Cinnamon as a complementary therapeutic approach for dysglycemia and dyslipidemia control in type 2 diabetes melli

# 6.Use results with Gemini

- https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini
- https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference

In [ ]:
import vertexai

from vertexai.generative_models import GenerativeModel, ChatSession

vertexai.init(project=PROJECT_ID, location=REGION)
model = GenerativeModel(model_name="gemini-1.0-pro-002")
chat = model.start_chat()

def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response)

## Try without context

Response will be based on the internal knowledge from Gemini

In [ ]:
prompt = query_text
print(get_chat_response(chat, prompt))

## Cinnamon and Prediabetes Management: Exploring Interactions with Other Interventions

Cinnamon has shown promise as a natural supplement for managing prediabetes, but understanding its interactions with other dietary and lifestyle interventions is crucial for optimizing its effectiveness and safety. Here's a breakdown of potential interactions:

**Dietary Interventions:**

* **Fiber-rich foods:** Combining cinnamon with high-fiber foods like fruits, vegetables, and legumes can enhance its blood sugar-lowering effects. Fiber slows down the digestion and absorption of carbohydrates, leading to a more gradual rise in blood sugar levels.
* **Low-glycemic index (GI) foods:** Pairing cinnamon with low-GI foods like whole grains, non-starchy vegetables, and lean protein can further improve blood sugar control. These foods cause a slower and smaller rise in blood sugar compared to high-GI foods.
* **Mediterranean diet:** Studies suggest that incorporating cinnamon into a Mediterranean diet,

## Try with context

Response will be based on the context provided to Gemini

In [ ]:
prompt = f"{query_text} in the context of:{neighbor_texts}"
print(get_chat_response(chat, prompt))

## Cinnamon and Prediabetes: Interactions with Other Interventions

As you mentioned, research including the study by Zelicha et al. suggests that cinnamon supplementation can positively impact blood sugar control in adults with prediabetes. However, it's important to consider how cinnamon interacts with other vital aspects of prediabetes management, including dietary and lifestyle interventions.

**Dietary Interactions:**

* **Fiber:** Studies suggest combining cinnamon with high-fiber foods may enhance its blood-sugar-lowering effects. Fiber slows down carbohydrate digestion and absorption, leading to a more gradual rise in blood sugar levels. This synergistic effect could be particularly beneficial for individuals with prediabetes.
* **Low-GI foods:** Pairing cinnamon with low-GI foods like whole grains, non-starchy vegetables, and lean protein could further improve blood sugar control. These foods cause a slower and smaller rise in blood sugar compared to high-GI foods, potentially

## Try only from context

Model instructed to answer only from the given context


In [ ]:
prompt = f"{query_text} Answer only from the following context:{neighbor_texts}"
print(get_chat_response(chat, prompt))

## Interactions of Cinnamon with Other Prediabetes Management Strategies (Based on Provided Context)

While the provided context revolves around the study by Zelicha et al. which investigates the effect of cinnamon on glycemic response in adults with prediabetes, it does not offer sufficient information to specifically address interactions with other dietary and lifestyle interventions. Therefore, a comprehensive response focusing solely on this context cannot be provided.

However, based on general knowledge and considering the study's conclusions, some potential interactions can be highlighted:

**Possible Synergistic Effects:**

* **High-fiber foods:** The slow digestion of fiber combined with cinnamon's blood sugar-lowering potential could lead to more sustained control of blood sugar levels.
* **Low-GI foods:** Pairing cinnamon with low-GI selections might further reduce blood sugar spikes after meals, enhancing glycemic management.
* **Regular physical activity:** Exercise improv

#7.Clean Up GCP ENV

In [ ]:
to_clean = True


## Delete Processor

- Processor location: https://console.cloud.google.com/ai/document-ai/locations/< us-or-eu >/processors?project=< your-project-id >

In [ ]:
from google.api_core import exceptions as gcp_exceptions

if to_clean:
    try:
        client.delete_processor(name=processor_name)
        print(f"Deleted Processor: {processor_name}")
    except gcp_exceptions.NotFound:
        print(f"Processor not found: {processor_name}")
else:
    print("clean_up parameter is set to False")

Deleted Processor: projects/314626027617/locations/us/processors/5d7d5f3c2d28335


## Delete Vector Search index

- https://console.cloud.google.com/vertex-ai/matching-engine/indexes?project=< your-project-id >

In [ ]:
if to_clean:
    try:
        vvs_index.delete()  # Set force=True to bypass the check for deployed indexes
        print(f"Deleted Matching Engine Index: {index.resource_name}")
    except gcp_exceptions.NotFound:
        print(f"Index not found: {index.resource_name}")
    except Exception as e:
        print(f"Error deleting index: {e}")
else:
    print("clean_up parameter is set to False")

INFO:google.cloud.aiplatform.base:Deleting MatchingEngineIndex : projects/314626027617/locations/northamerica-northeast1/indexes/5046494488701173760


Error deleting index: 400 The Index "projects/314626027617/locations/northamerica-northeast1/indexes/5046494488701173760" is deployed or being deployed at the following IndexEndpoint(s): projects/314626027617/locations/northamerica-northeast1/indexEndpoints/8155033762749480960.


## Delete the deployed endpoint

In [ ]:
from google.api_core import exceptions

if to_clean:
    try:
        # Undeploy the deployed index
        try:
            vvs_index_endpoint.undeploy_index(deployed_index_id=DEPLOYED_INDEX_ID)
            print(f"Undeployed index '{DEPLOYED_INDEX_ID}' from endpoint '{vvs_index_endpoint.name}'.")
        except exceptions.NotFound:
            print(f"Deployed index '{DEPLOYED_INDEX_ID}' not found in endpoint '{vvs_index_endpoint.name}'.")

        # Delete the index endpoint
        vvs_index_endpoint.delete()
        print(f"Deleted index endpoint: {vvs_index_endpoint.name}")

    except exceptions.NotFound:
        print(f"Index endpoint not found: {vvs_index_endpoint.name}")
else:
    print("clean_up parameter is set to False")

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Undeploying index MatchingEngineIndexEndpoint index_endpoint: projects/314626027617/locations/northamerica-northeast1/indexEndpoints/8155033762749480960
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Undeploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/314626027617/locations/northamerica-northeast1/indexEndpoints/8155033762749480960/operations/3997261878395928576
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:MatchingEngineIndexEndpoint index_endpoint Undeployed index. Resource name: projects/314626027617/locations/northamerica-northeast1/indexEndpoints/8155033762749480960
INFO:google.cloud.aiplatform.base:Deleting MatchingEngineIndexEndpoint : projects/314626027617/locations/northamerica-northeast1/indexEndpoints/8155033762749480960


Undeployed index 'deployed_index_07090208' from endpoint '8155033762749480960'.


INFO:google.cloud.aiplatform.base:Delete MatchingEngineIndexEndpoint  backing LRO: projects/314626027617/locations/northamerica-northeast1/indexEndpoints/8155033762749480960/operations/3244034840718213120
INFO:google.cloud.aiplatform.base:MatchingEngineIndexEndpoint deleted. . Resource name: projects/314626027617/locations/northamerica-northeast1/indexEndpoints/8155033762749480960


Deleted index endpoint: 8155033762749480960


## Delete bucket

- Bucket location: https://console.cloud.google.com/storage/

In [ ]:
if to_clean == True:
    # Delete the bucket
    ! gcloud storage rm --recursive gs://$BUCKET
else:
    print("delete_bucket parameter is set to False")

Removing objects:
Removing gs://rag-llmops-gcp-bucket/embeddings-data-07080211/batch_root/embeddings-07080211.json#1720406137994273...
Removing gs://rag-llmops-gcp-bucket/embeddings-data-07090054/batch_root/embeddings-07090054.json#1720486516784355...
Removing gs://rag-llmops-gcp-bucket/embeddings-data-07090208/batch_root/embeddings-07090208.json#1720490935672601...
Removing buckets:
Removing gs://rag-llmops-gcp-bucket/...
